##### -*- coding: utf8 -*-
### Information
- Created : 2024-03-18 11:00
- Updated : 2024-04-24 23:00
- Author : Okrie
- Description :   Airplane Price Information Test
- Version : 0.2

In [1]:
import pandas as pd
import requests as req
import time

Naver - https://flight.naver.com/flights/

#### Naver Info     
- seoul all , bk all
- round trip
- depart date = 2024.04.02
- arrival date = 2024.04.05
- adult = 2
- children = 2 ( 4, 10 )      
    
```code
https://flight.naver.com/flights/    
international/                  international    
ICN-BKK-20240402/               departure-arrival-depature_date
BKK-ICN-20240405                departure-arrival-arrival_date
?adults=2                       adult_count      
&child=2                        child_count    
&cabinclass=economy             carbin_class          
&children=1                     children_count         
&infant=1                       infant_count    OPTIONAL     
&isDirect=true                  직항=true OPTIONAL
fareType=Y                      Y 일반, P 프리미엄 일반, C 비지니스, F 일등석
```   
https://flight.naver.com/flights/international/ICN-BKK-20240402/BKK-ICN-20240405?adult=2&child=2&infant=1&isDirect=true&fareType=Y

In [2]:
departure = "ICN"
arrival = "KIX"
# KIX = 간사이 오사카
# FUK = 후쿠오카

# Test를 위한 날짜 자동 생성 (이전 날 나오지 않게 하기 위한 작업)
from datetime import datetime, timedelta 
# 현재 시간 + N day
def testTimeCal(plusDay=1):
    nTime = datetime.now() +timedelta(days=plusDay)
    return f'{nTime.year}-{str(nTime.month).zfill(2)}-{str(nTime.day).zfill(2)}'

departure_date = testTimeCal(2)
arrival_date = testTimeCal(3)

# Orignal Test
# departure_date = "2024-04-20"
# arrival_date = "2024-04-21"

adults_count = 2
children_count = 0
infant_count = 0

cabinclass = "economy"
direct = True


In [3]:
# url에 쓰일 수 있게 날짜 변환
def convertDateTouseUrl(date: str):
    return date.replace("-", "")

In [4]:
# children 수 체크
def hasChildrenCheck(children):
    if children > 0:
        return f'&child={children}'
    return ''

# infant 수 체크
def hasInfantCheck(infant):
    if infant > 0:
        return f'&infant={infant}'
    return ''

In [5]:
# 직항 유무
def isDirect(direct):
    return f'&isDirect=true' if direct else ''

In [6]:
# fare class 분류
def isFareType(carbinClass):
    if carbinClass == "economy": return "Y"
    if carbinClass == "premium economy": return "P"
    if carbinClass == "business": return "C"
    if carbinClass == "first": return "F"

In [7]:
# URL 작성
url = f'https://flight.naver.com/flights/international/'
url += f"{departure.upper()}-{arrival.upper()}-{convertDateTouseUrl(departure_date)}/"
url += f"{arrival.upper()}-{departure.upper()}-{convertDateTouseUrl(arrival_date)}"
url += f'?adults={adults_count}'
url += f"{hasChildrenCheck(children_count)}"
url += f"{hasInfantCheck(infant_count)}"
url += f'{isDirect(direct)}'
url += f'&fareType={isFareType(cabinclass)}'

In [8]:
url

'https://flight.naver.com/flights/international/ICN-KIX-20240426/KIX-ICN-20240427?adults=2&isDirect=true&fareType=Y'

In [9]:
# df columns 명명
columns = [
    'site',
    'dp_departure_airline',
    'dp_departure_img',
    'dp_departure_time',
    'dp_arrival_time',
    'dp_duration',
    'dp_departure_airport',
    'dp_arrival_airport',
    'dp_transit',
    'entry_departure_airline',
    'entry_departure_img',
    'entry_departure_time',
    'entry_arrival_time',
    'entry_duration',
    'entry_departure_airport',
    'entry_arrival_airport',
    'entry_transit',
    'ticket_info',
    'ticket_price',
    'link',
]

# DataFrame 생성
df = pd.DataFrame(columns=columns)

In [10]:
# api 주소
req_url = "https://airline-api.naver.com/graphql"

In [11]:
# 각 연령별 수 체크
adults = f'{adults_count if adults_count > 0 else 0}'
child = f'{children_count if children_count > 0 else 0}'
infant = f'{infant_count if infant_count > 0 else 0}'

In [12]:
# galileokey, travelbizkey 획득을 위한 post 요청
def firstGetKey():
    query = 'query getInternationalList($trip: InternationalList_TripType!, $itinerary: [InternationalList_itinerary]!, $adult: Int = '+adults+', $child: Int = '+child+', $infant: Int = '+infant+', $fareType: InternationalList_CabinClass!, $where: InternationalList_DeviceType = pc, $isDirect: Boolean = false, $stayLength: String, $galileoKey: String, $galileoFlag: Boolean = true, $travelBizKey: String, $travelBizFlag: Boolean = true) {        internationalList(            input: {trip: $trip, itinerary: $itinerary, person: {adult: $adult, child: $child, infant: $infant}, fareType: $fareType, where: $where, isDirect: $isDirect, stayLength: $stayLength, galileoKey: $galileoKey, galileoFlag: $galileoFlag, travelBizKey: $travelBizKey, travelBizFlag: $travelBizFlag}        ) {            galileoKey            galileoFlag            travelBizKey            travelBizFlag            totalResCnt            resCnt            results {            airlines            airports            fareTypes            schedules            fares            errors            carbonEmissionAverage {                directFlightCarbonEmissionItineraryAverage                directFlightCarbonEmissionAverage            }           }        }    }'
    
    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Referer": url,
    } 

    payload_first = {
        'operationName': "getInternationalList",
        "query": query,
        'variables': {
            'adult': int(adults),
            'child': int(child),
            'fareType': "Y",
            'galileoFlag': True,
            'galileoKey': "",
            'infant': int(infant),
            'isDirect': True,
            'itinerary': [
                {'departureAirport': departure.upper(), 'arrivalAirport': arrival.upper(), 'departureDate': convertDateTouseUrl(departure_date)},
                {'departureAirport': arrival.upper(), 'arrivalAirport': departure.upper(), 'departureDate': convertDateTouseUrl(arrival_date)}
            ],
            'stayLength': "",
            'travelBizFlag': False,
            'travelBizKey': "",
            'trip': "RT",
            'where': "pc",
        }
    }

    response = req.post(url=req_url, headers=headers, json=payload_first)
    return response

In [13]:
# 항공 데이터 post 요청
def getAirInfo(galileokey, travelbizkey):
    query = 'query getInternationalList($trip: InternationalList_TripType!, $itinerary: [InternationalList_itinerary]!, $adult: Int = '+adults+', $child: Int = '+child+', $infant: Int = '+infant+', $fareType: InternationalList_CabinClass!, $where: InternationalList_DeviceType = pc, $isDirect: Boolean = false, $stayLength: String, $galileoKey: String, $galileoFlag: Boolean = true, $travelBizKey: String, $travelBizFlag: Boolean = true) {        internationalList(            input: {trip: $trip, itinerary: $itinerary, person: {adult: $adult, child: $child, infant: $infant}, fareType: $fareType, where: $where, isDirect: $isDirect, stayLength: $stayLength, galileoKey: $galileoKey, galileoFlag: $galileoFlag, travelBizKey: $travelBizKey, travelBizFlag: $travelBizFlag}        ) {            galileoKey            galileoFlag            travelBizKey            travelBizFlag            totalResCnt            resCnt            results {            airlines            airports            fareTypes            schedules            fares            errors            carbonEmissionAverage {                directFlightCarbonEmissionItineraryAverage                directFlightCarbonEmissionAverage            }           }        }    }'

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
        "Referer": url,
    } 

    payload_first = {
        'operationName': "getInternationalList",
        "query": query,
        'variables': {
            'adult': int(adults),
            'child': int(child),
            'fareType': "Y",
            'galileoFlag': True,
            'galileoKey': galileokey,
            'infant': int(infant),
            'isDirect': True,
            'itinerary': [
                {'departureAirport': departure.upper(), 'arrivalAirport': arrival.upper(), 'departureDate': convertDateTouseUrl(departure_date)},
                {'departureAirport': arrival.upper(), 'arrivalAirport': departure.upper(), 'departureDate': convertDateTouseUrl(arrival_date)}
            ],
            'stayLength': "",
            'travelBizFlag': True,
            'travelBizKey': travelbizkey,
            'trip': "RT",
            'where': "pc",
        }
    }

    response = req.post(url=req_url, headers=headers, json=payload_first)
    return response

In [14]:
# keys 획득
resp = firstGetKey().json()
# 데이터 확인
# print(resp)

{'data': {'internationalList': {'galileoKey': '056_e7ee323e-7d20-4c78-86e7-b2ea29bd7cb9_582', 'galileoFlag': True, 'travelBizKey': 'BjD_LP4fhP', 'travelBizFlag': False, 'totalResCnt': 23, 'resCnt': 0, 'results': {'airlines': {}, 'airports': {}, 'fareTypes': {}, 'schedules': [], 'fares': {}, 'errors': [], 'carbonEmissionAverage': {'directFlightCarbonEmissionItineraryAverage': {'20240426ICNKIX': 20.27522641509434, '20240427KIXICN': 24.272462962962962}, 'directFlightCarbonEmissionAverage': 44.5476893780573}}}}}


In [15]:
# 받아온 Key 선언
galileo_key = resp['data']['internationalList']['galileoKey']
biz_key = resp['data']['internationalList']['travelBizKey']

In [16]:
# 앞선 post 요청 후 일정 딜레이 필요하여 넉넉하게 20초 지정
time.sleep(20)

# 받은 key로 post 항공사 데이터 요청
res = getAirInfo(galileokey=galileo_key, travelbizkey=biz_key)

In [17]:
api_data = res.json()

In [18]:
# tot - 총 개수, api_data - 항공 스케쥴 정보, fares - 가격 정보
tot = api_data['data']['internationalList']['totalResCnt']
api_data['data']['internationalList']['results']['schedules'][0]
fares = api_data['data']['internationalList']['results']['fares']

results = api_data['data']['internationalList']['results']

In [19]:
# 데이터 확인
# (fares['20240402ICNBKKLJ0001|20240405BKKICNLJ0002']['fare']['A01'])

In [20]:
# 항공 정보 및 항공사 이미지 주소 선언
airlines = api_data['data']['internationalList']['results']['airlines']
img_src = "https://vertical.pstatic.net/vertical/static/flight/airlines/"

In [21]:
# data 가공을 위한 선언
site = "naver"
ticket_info = []
ticket_tmp = 0
departure_id_list = []
dp_departure_id, entry_departure_id = [], []

In [22]:
# 항공 정보 별 묶음 및 id값 분리 저장
for price in fares:
    dp, entry = price.split("|")
    dp_departure_id.append(dp)
    entry_departure_id.append(entry)

In [23]:
# print(results['schedules'][0][dp_departure_id[0]]['detail'])
# print(len(dp_departure_id), len(entry_departure_id))

In [24]:
# fares[dp_departure_id[0]+"|"+entry_departure_id[0]]['fare']['A01'][0]

In [25]:
# percent-encoding -> decoding
from urllib.parse import unquote

In [26]:
# 출국
for i in range(len(dp_departure_id)):
    # 저장을 위한 임시 dict 타입 선언
    ticket_price = []

    dp_airline_data = results['schedules'][0][dp_departure_id[i]]
    entry_airline_data = results['schedules'][1][entry_departure_id[i]]

    # 출국
    dp_departure_airline = airlines[dp_airline_data['detail'][0]['av']] if "%" not in airlines[dp_airline_data['detail'][0]['av']] else unquote(airlines[dp_airline_data['detail'][0]['av']])
    dp_departure_img = img_src + dp_airline_data['detail'][0]['av'] + ".png"
    dp_departure_time = dp_airline_data['detail'][0]['sdt'][8:10] + ":" + dp_airline_data['detail'][0]['sdt'][10:12]
    dp_arrival_time = dp_airline_data['detail'][0]['edt'][8:10] + ":" + dp_airline_data['detail'][0]['edt'][10:12]
    dp_duration = dp_airline_data['detail'][0]['jt'][:2] + ":" + dp_airline_data['detail'][0]['jt'][2:]
    dp_departure_airport = dp_airline_data['detail'][0]['sa']
    dp_arrival_airport = dp_airline_data['detail'][0]['ea']
    dp_transit = "직항" if "+" not in dp_departure_id else f"{dp_departure_id.count('+')}회 이상 경유"
    
    # 입국
    entry_departure_airline = airlines[entry_airline_data['detail'][0]['av']] if "%" not in airlines[entry_airline_data['detail'][0]['av']] else unquote(airlines[entry_airline_data['detail'][0]['av']])
    entry_departure_img = img_src + entry_airline_data['detail'][0]['av'] + ".png"
    entry_departure_time = entry_airline_data['detail'][0]['sdt'][8:10] + ":" + entry_airline_data['detail'][0]['sdt'][10:12]
    entry_arrival_time = entry_airline_data['detail'][0]['edt'][8:10] + ":" + entry_airline_data['detail'][0]['edt'][10:12]
    entry_duration = entry_airline_data['detail'][0]['jt'][:2] + ":" + entry_airline_data['detail'][0]['jt'][2:]
    entry_departure_airport = entry_airline_data['detail'][0]['sa']
    entry_arrival_airport = entry_airline_data['detail'][0]['ea']
    entry_transit = "직항" if "+" not in entry_departure_id else f"{entry_departure_id.count('+')}회 이상 경유"
    
    ticket_info = f"총 {len(fares[dp_departure_id[i]+'|'+entry_departure_id[i]]['fare']['A01'])}개의 항공권"

    # 가격 계산
    for j in range(len(fares[dp_departure_id[i]+"|"+entry_departure_id[i]]['fare']['A01'])):
        # 어른 가격
        adults_prices = [\
                int(price)
                for price in fares[dp_departure_id[i]+"|"+entry_departure_id[i]]['fare']['A01'][j]['Adult'].values() 
                ]
        # 아이 가격
        childs_prices = [\
                int(price)
                for price in fares[dp_departure_id[i]+"|"+entry_departure_id[i]]['fare']['A01'][j]['Child'].values() 
                ]
        
        # 총 가격
        if sum(adults_prices[1:]) <= 0:
            ticket_price.append(sum(adults_prices) + sum(childs_prices))
        else:
            ticket_price.append(sum(adults_prices[1:]) + sum(childs_prices[1:]))
    
    
    link = [\
            fares[dp_departure_id[i]+"|"+entry_departure_id[i]]['fare']['A01'][j]['ReserveParameter']['#cdata-section']
                for j in range(len(fares[dp_departure_id[i]+"|"+entry_departure_id[i]]['fare']['A01']))
            ]

    one_data = {
        'site' : site,
        'dp_departure_airline' : dp_departure_airline,
        'dp_departure_img' : dp_departure_img,
        'dp_departure_time' : dp_departure_time,
        'dp_arrival_time' : dp_arrival_time,
        'dp_duration' : dp_duration,
        'dp_departure_airport' : dp_departure_airport,
        'dp_arrival_airport' : dp_arrival_airport,
        'dp_transit' : dp_transit,
        'entry_departure_airline' : entry_departure_airline,
        'entry_departure_img' : entry_departure_img,
        'entry_departure_time' : entry_departure_time,
        'entry_arrival_time' : entry_arrival_time,
        'entry_duration' : entry_duration,
        'entry_departure_airport' : entry_departure_airport,
        'entry_arrival_airport' : entry_arrival_airport,
        'entry_transit' : entry_transit,
        'ticket_info' : ticket_info,
        'ticket_price' : ticket_price,
        'link' : link,
    }

    df = pd.concat([df, pd.DataFrame([one_data])], ignore_index=True)

In [27]:
# df = pd.concat([df, pd.DataFrame([one_data])], ignore_index=True)

df

,site,dp_departure_airline,dp_departure_img,dp_departure_time,dp_arrival_time,dp_duration,dp_departure_airport,dp_arrival_airport,dp_transit,entry_departure_airline,entry_departure_img,entry_departure_time,entry_arrival_time,entry_duration,entry_departure_airport,entry_arrival_airport,entry_transit,ticket_info,ticket_price,link
0,naver,진에어,https://vertical.pstatic.net/vertical/static/f...,16:05,18:00,01:55,ICN,KIX,직항,진에어,https://vertical.pstatic.net/vertical/static/f...,19:10,21:00,01:50,KIX,ICN,직항,총 12개의 항공권,"[302200, 310500, 103500, 310300, 194600, 10350...",[https://naver.hanatour.com/trp/air/CHPC0AIR02...
1,naver,에어서울,https://vertical.pstatic.net/vertical/static/f...,13:15,15:15,02:00,ICN,KIX,직항,에어서울,https://vertical.pstatic.net/vertical/static/f...,19:00,21:00,02:00,KIX,ICN,직항,총 5개의 항공권,"[318900, 322900, 327700, 424000, 127200]",[https://naver.hanatour.com/trp/air/CHPC0AIR02...
2,naver,진에어,https://vertical.pstatic.net/vertical/static/f...,14:40,16:30,01:50,ICN,KIX,직항,진에어,https://vertical.pstatic.net/vertical/static/f...,19:10,21:00,01:50,KIX,ICN,직항,총 12개의 항공권,"[310900, 319500, 103500, 319300, 194600, 10350...",[https://naver.hanatour.com/trp/air/CHPC0AIR02...
3,naver,진에어,https://vertical.pstatic.net/vertical/static/f...,16:05,18:00,01:55,ICN,KIX,직항,이스타항공,https://vertical.pstatic.net/vertical/static/f...,18:00,19:55,01:55,KIX,ICN,직항,총 3개의 항공권,"[330200, 420400, 120100]",[https://naver.hanatour.com/trp/air/CHPC0AIR02...
4,naver,에어서울,https://vertical.pstatic.net/vertical/static/f...,16:00,18:00,02:00,ICN,KIX,직항,에어서울,https://vertical.pstatic.net/vertical/static/f...,19:00,21:00,02:00,KIX,ICN,직항,총 5개의 항공권,"[328700, 332900, 337700, 433700, 127200]",[https://naver.hanatour.com/trp/air/CHPC0AIR02...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801,naver,진에어,https://vertical.pstatic.net/vertical/static/f...,16:05,18:00,01:55,ICN,KIX,직항,피치항공,https://vertical.pstatic.net/vertical/static/f...,18:05,20:05,02:00,KIX,ICN,직항,총 1개의 항공권,[70300],[http://kr.trip.com/flights/Transfer?flighttyp...
802,naver,진에어,https://vertical.pstatic.net/vertical/static/f...,16:05,18:00,01:55,ICN,KIX,직항,피치항공,https://vertical.pstatic.net/vertical/static/f...,19:55,21:50,01:55,KIX,ICN,직항,총 1개의 항공권,[70300],[http://kr.trip.com/flights/Transfer?flighttyp...
803,naver,진에어,https://vertical.pstatic.net/vertical/static/f...,16:05,18:00,01:55,ICN,KIX,직항,대한항공,https://vertical.pstatic.net/vertical/static/f...,11:55,13:50,01:55,KIX,GMP,직항,총 1개의 항공권,[124000],[http://kr.trip.com/flights/Transfer?flighttyp...
804,naver,진에어,https://vertical.pstatic.net/vertical/static/f...,16:05,18:00,01:55,ICN,KIX,직항,아시아나항공,https://vertical.pstatic.net/vertical/static/f...,20:40,22:25,01:45,KIX,GMP,직항,총 1개의 항공권,[124600],[http://kr.trip.com/flights/Transfer?flighttyp...
